<a href="https://colab.research.google.com/github/saeedmoazami/Spherical_Mobile_Robot/blob/master/Spherical_Robot_GYM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%cd /content/gdrive/My Drive/SR

In [0]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import time
import random
# import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

In [0]:
# env = gym.make('CartPole-v0') # initialise environment


# state_size = env.observation_space.shape[0]
state_size = 14

# action_size = env.action_space.n
action_size = 9

batch_size = 32

total_episodes = 100000 # n games we want agent to play (default 1001)

total_time_step = 50000

target_position = [5,1]

print_interval = 100
# Libra = Spherical_Robot(target_position)

In [0]:
import gym
import math
import numpy as np

from gym import spaces, logger
from gym.utils import seeding

from matplotlib import pyplot as plt



class NormaEnv():

    def __init__(self, ):
#         Environment Properties
        self.gravity   =  9.8
        self.time_step = 0.01

#         Norma properties
        self.mass_sphere      =  3
        self.radius           =  0.2
        self.pendulum_length  =  0.15
        self.mass_slider      =  1
        self.mass_pendulum    =  1
        self.mass_shaft       =  0.6
        self.slider_init_pos  =  0
        self.pend_init_pose   =  0

        self.I_shaft   =  (1/12)  *  self.mass_shaft  *  (2 * self.radius)**2
        self.I_sphere  =  (2/3)   *  self.mass_sphere *  self.radius**2
        
#         self.seed()
        self.slider_limit = 0.2
        self.target_reached_distance = 0.05
        
        self.FORCE = 20
        self.TORQUE = 2
        
        
        
        
        
        
        
#         self.target_x = target_init_x
#         self.target_y = target_init_y

# ISx0 = 0.1;       ISy0 = 0.1;         ISz0 = 0.1; 

# ISx = ISx0;       ISy = ISy0;   ISz = ISz0;
        self.I_pendulum  =  self.mass_pendulum  *  (self.pendulum_length**2)

# C2 = 0;
# C3 = 0;


#         high = np.array([1., 1., self.max_speed], dtype=np.float32)
#         self.action_space = spaces.Box(low=-self.max_torque, high=self.max_torque, shape=(1,), dtype=np.float32)
#         self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)

    def distance_func(self):
        """
        This function does not update self.distance to keep track the changes
        """
        distance = np.sqrt((self.state[0] - self.state[2])**2 + (self.state[1] - self.state[3])**2)
        
        return distance
    


#     def seed(self, seed=None):
#         self.np_random, seed = seeding.np_random(seed)
#         return [seed]
    
    
#     def reward_func(self):
#         # Should be distance change
#         distance = math.sqrt( (self.state[0]-self.target_x)**2 + (self.state[1]-self.target_y)**2 )
            
#         return -distance

    def reward_function(self):
        
        old_distance = self.distance
        
        new_distance = self.distance_func()

        if  new_distance < self.target_reached_distance:

            self.target_reached = True

            print("TARGET REACHED _______________________________________________________________________/n/n/n")

            self.reward = +10

        else:

            self.reward = (old_distance - new_distance) * 1000
        
        self.distance = new_distance
        
        return self.reward
    
    
    
    def step(self, action, Discrete = True):
        
#         force  =  self.FORCE
#         torque = self.TORQUE

        if not Discrete: # Continuous
        
            force, torque  = action[0:2]

        else:
            
            force  = self.FORCE       
            torque = self.TORQUE
            
            if action == 0:
                force = 0
                torque = 0
                
            elif action == 1:
                force = 0
                torque = self.TORQUE

            elif action == 2:
                force = 0
                torque = - self.TORQUE

            elif action == 3:                
                force = self.FORCE
                torque = 0

            elif action == 4:                
                force = self.FORCE
                torque = self.TORQUE

            elif action == 5:                
                force = self.FORCE
                torque = -self.TORQUE

            elif action == 6:                
                force = -self.FORCE
                torque = 0

            elif action == 7:                
                force = -self.FORCE
                torque = self.TORQUE

            elif action == 8:                
                force = -self.FORCE
                torque = -self.TORQUE

            else:
                
                print("INVALID ACTION")
        
        load_matrix = np.reshape([0, torque, 0, force], (4))
        
        x, y, target_x, target_y, theta, alpha, phi, delta, psi, theta_dot, alpha_dot, phi_dot, delta_dot, psi_dot  = self.state
        
        m11 = self.pendulum_length**2 * self.mass_pendulum * math.cos(alpha-theta)**2 * (1+math.cos(phi)**4)
        - 2 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos(phi) * math.sin(phi)**3 
        + 2 * self.pendulum_length * self.mass_pendulum * math.cos(alpha-theta) * math.cos(phi) * (-self.radius+self.pend_init_pose * math.cos(phi)**2 * math.sin(phi)) 
        + self.pendulum_length**2 * self.mass_pendulum * math.sin(phi)**2 * (math.sin(alpha-theta)**2 
        + math.sin(phi)**2) + math.cos(phi)**2 * (self.I_sphere+(self.mass_pendulum+self.mass_slider+self.mass_sphere)*self.radius**2 
        + self.pendulum_length**2 * self.mass_pendulum * math.sin(alpha-theta)**2 
        + (self.I_shaft+2*(self.pend_init_pose**2 * self.mass_pendulum+self.mass_slider*(self.slider_init_pos+delta)**2))*math.sin(phi)**2)
        
        m12 = -self.mass_pendulum * (self.pendulum_length**2 * math.cos(alpha-theta)**2 * (1+math.cos(phi)**4)
        -2 * self.pendulum_length * self.pend_init_pose * math.cos(phi)*math.sin(phi)**3+self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)*(-self.radius+2 * self.pend_init_pose * math.cos(phi)**2 * math.sin(phi))+self.pendulum_length**2 * math.sin(phi)**2*(math.sin(alpha-theta)**2+math.sin(phi)**2)+math.cos(phi)**2*(self.pendulum_length**2 * math.sin(alpha-theta)**2+2 * self.pend_init_pose**2 * math.sin(phi)**2))
        
        m13 = self.pendulum_length * self.mass_pendulum * math.sin(alpha-theta)*(self.pend_init_pose-self.radius * math.sin(phi))
        m14 = 0
        c11 = math.cos(phi)*(self.pendulum_length * self.mass_pendulum*(alpha_dot-theta_dot)*math.sin(alpha-theta)*(self.radius-math.cos(phi)**2*(self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)+self.pend_init_pose * math.sin(phi)))+self.mass_slider*(self.slider_init_pos+delta)*delta_dot * math.sin(phi)*math.sin(2 * phi))+1/8 * phi_dot*(8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos((alpha-theta)/2)**2 * math.cos(4 * phi)+8 * self.pendulum_length * self.mass_pendulum*(-self.pend_init_pose * math.cos(2 * phi)*math.sin((alpha-theta)/2)**2+self.radius * math.cos(alpha-theta)*math.sin(phi))-2*(2 * self.I_sphere-self.pendulum_length**2 * self.mass_pendulum+2*(self.mass_pendulum+self.mass_slider+self.mass_sphere)*self.radius**2+self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*math.sin(2 * phi)+(2 * self.I_shaft-3 * self.pendulum_length**2 * self.mass_pendulum+4 * self.pend_init_pose**2 * self.mass_pendulum+4 * self.mass_slider*(self.slider_init_pos+delta)**2-self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*math.sin(4 * phi));
        c12 = self.pendulum_length * self.mass_pendulum * math.cos(phi)*(alpha_dot-theta_dot)*math.sin(alpha-theta)*(-self.radius+self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)**3+self.pend_init_pose * math.cos(phi)**2 * math.sin(phi))+1/4 * self.mass_pendulum * phi_dot*(8 * self.pendulum_length**2 * math.cos(alpha-theta)**2 * math.cos(phi)**3 * math.sin(phi)-2 * self.pendulum_length * math.cos(alpha-theta)*(self.pend_init_pose*(math.cos(2 * phi)+math.cos(4 * phi))+self.radius * math.sin(phi))+4 * self.pendulum_length * math.sin(phi)*(-self.pendulum_length * math.cos(phi)+self.pend_init_pose * math.sin(3 * phi))+(self.pendulum_length**2-2 * self.pend_init_pose**2)*math.sin(4 * phi))
        c13 = 1/8*(8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos((alpha-theta)/2)**2 * math.cos(4 * phi)*(-alpha_dot+theta_dot)+8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos(2 * phi)*(alpha_dot-theta_dot)*math.sin((alpha-theta)/2)**2-8 * self.pendulum_length**2 * self.mass_pendulum * phi_dot * math.sin(2*(alpha-theta))-4 * self.pendulum_length * self.mass_pendulum * self.radius * math.cos(alpha-theta)*(alpha_dot-2 * theta_dot)*math.sin(phi)+2*(-(2 * self.I_sphere-self.pendulum_length**2 * self.mass_pendulum+2*(self.mass_pendulum+self.mass_slider+self.mass_sphere)*self.radius**2+self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*theta_dot-2 * self.pendulum_length**2 * self.mass_pendulum * alpha_dot * math.sin(alpha-theta)**2)*math.sin(2 * phi)+(self.mass_pendulum*(3 * self.pendulum_length**2-4 * self.pend_init_pose**2+self.pendulum_length**2 * math.cos(2*(alpha-theta)))*alpha_dot+(2 * self.I_shaft-3 * self.pendulum_length**2 * self.mass_pendulum+4 * self.pend_init_pose**2 * self.mass_pendulum+4 * self.mass_slider*(self.slider_init_pos+delta)**2-self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*theta_dot)*math.sin(4 * phi))
        c14 = 2 * self.mass_slider*(self.slider_init_pos+delta)*math.cos(phi)**2 * theta_dot * math.sin(phi)**2
        m21 = m12
        m22 =  self.mass_pendulum*(self.pendulum_length**2 * math.cos(alpha-theta)**2+self.pendulum_length**2 * math.cos(phi)**2 * math.sin(alpha-theta)**2+self.pendulum_length**2 * math.sin(alpha-theta)**2 * math.sin(phi)**2+math.sin(phi)**2*(self.pend_init_pose * math.cos(phi)-self.pendulum_length * math.sin(phi))**2+math.cos(phi)**2*(self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)+self.pend_init_pose * math.sin(phi))**2)
        m23=-self.pendulum_length * self.mass_pendulum * math.sin(alpha-theta)*(self.pend_init_pose-self.radius * math.sin(phi))
        m24 = 0
        c21 = self.pendulum_length * self.mass_pendulum * math.cos(phi)**3*(alpha_dot-theta_dot)*math.sin(alpha-theta)*(self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)+self.pend_init_pose * math.sin(phi))+1/4 * self.mass_pendulum * phi_dot*(8 * self.pendulum_length**2 * math.cos(alpha-theta)**2 * math.cos(phi)**3 * math.sin(phi)-2 * self.pendulum_length * math.cos(alpha-theta)*(self.pend_init_pose*(math.cos(2 * phi)+math.cos(4 * phi))+self.radius * math.sin(phi))+4 * self.pendulum_length * math.sin(phi)*(-self.pendulum_length * math.cos(phi)+self.pend_init_pose * math.sin(3 * phi))+(self.pendulum_length**2-2 * self.pend_init_pose**2)*math.sin(4 * phi));
        c22=-self.pendulum_length * self.mass_pendulum * math.cos(phi)**3*(alpha_dot-theta_dot)*math.sin(alpha-theta)*(self.pendulum_length * math.cos(alpha-theta)*math.cos(phi)+self.pend_init_pose * math.sin(phi))-1/4 * self.mass_pendulum * phi_dot*(-4 * self.pendulum_length * math.cos(phi)*(self.pend_init_pose * math.cos(alpha-theta)*math.cos(3 * phi)+self.pendulum_length * math.sin(phi)-2 * self.pendulum_length * math.cos(alpha-theta)**2 * math.cos(phi)**2 * math.sin(phi))+4 * self.pendulum_length * self.pend_init_pose * math.sin(phi)*math.sin(3 * phi)+(self.pendulum_length**2-2 * self.pend_init_pose**2)*math.sin(4 * phi));
        c23 = 1/8 * self.mass_pendulum*(8 * self.pendulum_length * self.pend_init_pose * math.cos((alpha-theta)/2)**2 * math.cos(4 * phi)*(alpha_dot-theta_dot)+8 * self.pendulum_length * self.pend_init_pose * math.cos(2 * phi)*(-alpha_dot+theta_dot)*math.sin((alpha-theta)/2)**2+8 * self.pendulum_length**2 * phi_dot * math.sin(2*(alpha-theta))-4 * self.pendulum_length * self.radius * math.cos(alpha-theta)*theta_dot * math.sin(phi)+4 * self.pendulum_length**2*(alpha_dot-theta_dot)*math.sin(alpha-theta)**2 * math.sin(2 * phi)-(3 * self.pendulum_length**2-4 * self.pend_init_pose**2+self.pendulum_length**2 * math.cos(2*(alpha-theta)))*(alpha_dot-theta_dot)*math.sin(4 * phi));
        c24 = 0
        m31 = m13
        m32 = m23
        m33=(self.I_shaft+self.I_sphere+self.pendulum_length**2 * self.mass_pendulum+2 * self.pend_init_pose**2 * self.mass_pendulum+2 * self.slider_init_pos**2 * self.mass_slider+(self.mass_pendulum+self.mass_slider+self.mass_sphere)*self.radius**2+4 * self.slider_init_pos * self.mass_slider * delta+2 * self.mass_slider * delta**2+self.pendulum_length * self.mass_pendulum*(self.pendulum_length * math.cos(2*(alpha-theta))-2 * self.radius * math.cos(alpha-theta)*math.cos(phi))+2 * self.radius*(-self.pend_init_pose * self.mass_pendulum+self.mass_slider*(self.slider_init_pos+delta))*math.sin(phi))
        m34=-self.mass_slider * self.radius * math.cos(phi)
        c31 = self.pendulum_length * self.mass_pendulum*(2 * self.pendulum_length * math.cos(alpha-theta)-self.radius * math.cos(phi))*phi_dot * math.sin(alpha-theta)+1/4 * self.mass_pendulum * alpha_dot*(2 * self.pendulum_length*(-4 * self.pendulum_length * math.cos(alpha-theta)**2 * math.cos(phi)**3 * math.sin(phi)+math.cos(alpha-theta)*(self.pend_init_pose*(2+math.cos(2 * phi)+math.cos(4 * phi))-self.radius * math.sin(phi))+self.pendulum_length * math.sin(2 * phi)-2 * self.pend_init_pose * math.sin(phi)*math.sin(3 * phi))-(self.pendulum_length**2-2 * self.pend_init_pose**2)*math.sin(4 * phi))+1/8 * theta_dot*(-8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos(alpha-theta)-8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos((alpha-theta)/2)**2 * math.cos(4 * phi)+8 * self.pendulum_length * self.pend_init_pose * self.mass_pendulum * math.cos(2 * phi)*math.sin((alpha-theta)/2)**2+2*(2 * self.I_sphere-self.pendulum_length**2 * self.mass_pendulum+2*(self.mass_pendulum+self.mass_slider+self.mass_sphere)*self.radius**2+self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*math.sin(2 * phi)-(2 * self.I_shaft-3 * self.pendulum_length**2 * self.mass_pendulum+4 * self.pend_init_pose**2 * self.mass_pendulum+4 * self.mass_slider*(self.slider_init_pos+delta)**2-self.pendulum_length**2 * self.mass_pendulum * math.cos(2*(alpha-theta)))*math.sin(4 * phi))
        c32 = 1/8 * self.mass_pendulum*(8 * self.pendulum_length * self.pend_init_pose * math.cos((alpha-theta)/2)**2 * math.cos(4 * phi)*(-alpha_dot+theta_dot)+8 * self.pendulum_length * self.pend_init_pose * math.cos(2 * phi)*(alpha_dot-theta_dot)*math.sin((alpha-theta)/2)**2+8 * self.pendulum_length * self.radius * math.cos(phi)*phi_dot * math.sin(alpha-theta)+8 * self.pendulum_length * math.cos(alpha-theta)*(-self.pend_init_pose * alpha_dot+self.pend_init_pose * theta_dot-2 * self.pendulum_length * phi_dot * math.sin(alpha-theta))+4 * self.pendulum_length * self.radius * math.cos(alpha-theta)*(2 * alpha_dot-theta_dot)*math.sin(phi)+4 * self.pendulum_length**2*(-alpha_dot+theta_dot)*math.sin(alpha-theta)**2 * math.sin(2 * phi)+(3 * self.pendulum_length**2-4 * self.pend_init_pose**2+self.pendulum_length**2 * math.cos(2*(alpha-theta)))*(alpha_dot-theta_dot)*math.sin(4 * phi))
        c33  =  self.pendulum_length * self.mass_pendulum*(-2 * self.pendulum_length * math.cos(alpha-theta)+self.radius * math.cos(phi))*(alpha_dot-theta_dot)*math.sin(alpha-theta)+self.mass_slider * delta_dot * (2*(self.slider_init_pos+delta)+self.radius * math.sin(phi))+self.radius * phi_dot*((-self.pend_init_pose * self.mass_pendulum+self.mass_slider*(self.slider_init_pos+delta))*math.cos(phi)+self.pendulum_length * self.mass_pendulum * math.cos(alpha-theta)*math.sin(phi))
        c34 = self.mass_slider * phi_dot*(2*(self.slider_init_pos+delta)+self.radius * math.sin(phi))
        m41 = 0
        m42 = 0
        m43 = m34
        m44 = self.mass_slider

        c41=-2 * self.mass_slider*(self.slider_init_pos+delta)*math.cos(phi)**2 * theta_dot * math.sin(phi)**2
        c42 = 0
        c43=-2 * self.mass_slider*(self.slider_init_pos+delta)*phi_dot
        c44 = 0
        g1 = - self.gravity * self.pendulum_length * self.mass_pendulum * math.cos(phi)*math.sin(alpha-theta)
        g2 =   self.gravity * self.pendulum_length * self.mass_pendulum * math.cos(phi)*math.sin(alpha-theta)
        g3_balance  =  (-self.pend_init_pose * self.mass_pendulum+self.mass_slider*(self.slider_init_pos+delta))
        g3 =  self.gravity*(  g3_balance  * math.cos(phi)   +    self.pendulum_length * self.mass_pendulum * math.cos(alpha-theta)*math.sin(phi))
        g4 =  self.gravity * self.mass_slider * math.sin(phi)
        
        #         Inertia Matrix 
        M = [m11, m12, m13, m14, m21, m22, m23, m24, m31, m32, m33, m34, m41, m42, m43, m44]

        M = np.reshape(M,(4,4)).astype(float)
        
#         Coriolis Temrs 
        C = [c11, c12, c13, c14, c21, c22, c23, c24, c31, c32, c33, c34, c41, c42, c43, c44]
        C = np.reshape(C,(4,4))
        
#         Gravitational terms
        G = [g1,g2,g3,g4]
        G = np.reshape(G,(4))
        
#         M*Q_d_dot + C*Q_dot + G = Load 


#         generalized_coordinates

        Q = np.reshape([theta, alpha, phi, delta], (4))

        Q_dot = np.reshape(([theta_dot, alpha_dot, phi_dot, delta_dot]), (4))


        M_inverse = np.linalg.inv(M)
        
        Q_ddot = np.dot(M_inverse , (load_matrix - (np.dot(C,Q_dot) + G)))
        
        Q_dot = Q_ddot * self.time_step + Q_dot
        
        Q = Q_dot * self.time_step + Q
        
        theta, alpha, phi, delta = Q
        
        theta_dot, alpha_dot, phi_dot, delta_dot = Q_dot
        
        x_dot = self.radius * (theta_dot * math.cos(phi) * math.cos(psi) + phi_dot * math.sin(psi))
        
        y_dot = self.radius * (theta_dot * math.cos(phi) * math.sin(psi) - phi_dot * math.cos(psi))
        
        psi_dot   =  -theta_dot * math.sin(phi)

        x   = x_dot * self.time_step + x
        
        y   = y_dot * self.time_step + y
        
        psi = psi_dot * self.time_step + psi
        
        state_list = [x, y, target_x, target_y,  theta, alpha, phi, delta, psi, theta_dot, alpha_dot, phi_dot, delta_dot, psi_dot]
 
        self.state = np.reshape(state_list, (state_size))     
        
#         self.observation = np.reshape([(x - self.target_x), (y - self.target_y), theta, delta], (1,4))
    
        self.observation = self.state
        
        if (np.abs(delta)> self.slider_limit):
            self.done = True
            self.reward = -10
        else:
            self.reward = self.reward_function()

        return self.observation, self.reward , self.done, {}
    
    
    def target_update(self, target_position):
        self.state[2:4] = np.array(target_position)
            
            
    def reset(self, target_position):
        
        self.done =False
        
        self.target_reached = False
        
        # self.state = np.zeros(state_size, dtype='float')
        
        self.state = ((np.random.rand(state_size) * 0.01) - 0.005).astype(float)

        self.target_update(target_position)
        
        """
        This is the order of parameters considered as state:
        
        
        x, y, target_x, target_y, theta, alpha, phi, delta, psi, theta_dot, alpha_dot, phi_dot, delta_dot, psi_dot  = self.state
        
        """

        self.distance = self.distance_func()

#         self.steps_beyond_done = None
        return self.state
        


In [0]:
output_dir = 'model_output/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

env = NormaEnv()

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=200000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.9995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.alpha = 0.01
        self.model = self._build_model() # private method 
        # self.load(output_dir + "weights.hdf5")
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        print("Building Model ------------------------------------------------------------")
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(64, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                # target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))

            # Q_values = self.model.predict(state) # approximately map current state to future discounted reward


            # Q_values[0][action] = (Q_values[0][action]) * (1-self.alpha) + (self.alpha * target)


            # self.model.fit(state, Q_values, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat

                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat



        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        print("Model Loaded ----------------------------------------------------------------------------\n")
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

agent = DQNAgent(state_size, action_size) # initialise agent

done = False

average_rewards = []
total_time_steps = []


for e in range(total_episodes): # iterate over new episodes of the game
    episode_reward = 0
    # print("Episode:  ", e)
    state = env.reset(target_position) # reset state at start of each new episode of the game
    
    
    state = np.reshape(state, [1, state_size])
    
    time_step = 0
    for time_step in range(total_time_step):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position  
        episode_reward += reward
        # print(time_step)     

        # reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        
#         if len(agent.memory) > batch_size and time_step % 100 == 0:

        # print("We are here")
           # train the agent by replaying the experiences of the episode
        
        
        if done: # episode ends if agent drops pole or we reach timestep 5000
            # print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
            #       .format(e, n_episodes, time_step, agent.epsilon))
            agent.replay(batch_size)
            break # exit loop
    
    episode_average_reward = episode_reward/time_step
    average_rewards += [episode_average_reward]
    total_time_steps += [time_step]
    if e % print_interval ==0:
        print("Episode: ", e, "           Final time_step : " , time_step, "      Final Distance :", env.distance)
        print("Average reward = ", episode_average_reward,"\n")
        agent.save(output_dir + "weights.hdf5")
        print("Model saved at episode: ", e, "-----------------------------------------------------")
        

# saved agents can be loaded with agent.load("./path/filename.hdf5")